In [1]:
# 예제1
def deco(func):
  def inner():
    print('running inner()')
  return inner

@deco
def target():
  print('running target()')

target()

running inner()


In [2]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
    result =  func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned {result}")
    return result
  return wrapper

@logger
def add(a, b):
  return a + b

In [5]:
# 예제2
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__=='__main__':
  main()

running register(<function f1 at 0x78e83e6f0b80>)
running register(<function f2 at 0x78e83e6f0d60>)
running main()
registry -> [<function f1 at 0x78e83e6f0b80>, <function f2 at 0x78e83e6f0d60>]
running f1()
running f2()
running f3()


In [6]:
# 예제3
promos = []

def promotion(promo_func):
  """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_tiem_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  """최대로 할인받을 금액을 반환한다."""
  return max(promo(order) for promo in promos)

In [7]:
# 예제4
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [8]:
# 예제5
b = 6
def f2(a):
  print(a)
  print(b)
  b=9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [9]:
# 예제6
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [10]:
b

9

In [11]:
f3(3)

3
9


In [12]:
b=30
f3(3)

3
30


In [13]:
# 예제7
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [15]:
# 예제8
def maek_averager():
  series = []

  def averager(new_balue):
    series.append(new_balue)
    total = sum(series)
    return total/len(series)

  return averager

In [17]:
avg = maek_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

In [20]:
# 예제9
avg.__code__.co_varnames

('new_balue', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x78e83d4ec730: list object at 0x78e83d704800>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [24]:
# 예제10
def maek_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [25]:
avg = maek_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [26]:
# 예제11
def maek_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [27]:
avg = maek_averager()
avg(10)

10.0

In [28]:
# 예제12
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [29]:
# 예제13
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12310497s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000049s] factorial(1) -> 1
[0.00001450s] factorial(2) -> 2
[0.00002145s] factorial(3) -> 6
[0.00002792s] factorial(4) -> 24
[0.00003457s] factorial(5) -> 120
[0.00004318s] factorial(6) -> 720
6! = 720


In [30]:
# 예제14
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.item())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [31]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12314057s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000048s] factorial(1) -> 1
[0.00001264s] factorial(2) -> 2
[0.00001884s] factorial(3) -> 6
[0.00002384s] factorial(4) -> 24
[0.00002933s] factorial(5) -> 120
[0.00003600s] factorial(6) -> 720
6! = 720
